In [55]:
import pandas as pd
import sqlite3 as sql
import pyreadstat
from tqdm import tqdm 
import os
import wrds
import numpy as np
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [56]:
# we don't aggregate permno in this study 
class stock:
    def __init__(self,begin,end,Conn):
        self.begin=begin
        self.end=end
        self.Conn=Conn
    
    def get_monthly_data(self):
        
        crsp_m=self.Conn.raw_sql(f''' select a.permco, a.permno, a.date, a.PRC, a.shrout, a.ret, a.vol, b.shrcd, b.exchcd, b.siccd, b.ticker
                            from crsp.msf as a
                            left join crsp.msenames as b
                            on a.permno=b.permno 
                            and a.date>=b.namedt
                            and b.nameendt>=a.date
                            where a.date between '{self.begin}' and '{self.end}'
                            AND shrcd in ('10','11')
                                                ''',date_cols=['date'])     
        
        # change variable format to int
        crsp_m= crsp_m.dropna(subset=['permco','permno','shrcd','exchcd','siccd'])

        crsp_m[['permco','permno','shrcd','exchcd','siccd']]=\
            crsp_m[['permco','permno','shrcd','exchcd','siccd']].astype(int)

        #adjust the date
        crsp_m['date']=crsp_m['date']+MonthEnd(0)
        crsp_m['year']=crsp_m['date'].dt.year
        crsp_m['day']=crsp_m['date'].dt.day
        crsp_m['month']=crsp_m['date'].dt.month
        crsp_m['day']=crsp_m['date'].dt.day

        # add delisting return
        dlret = self.Conn.raw_sql("""
                             select permno, dlret, dlstdt 
                             from crsp.msedelist
                             """, date_cols=['dlstdt'])

        dlret.permno=dlret.permno.astype(int)
        dlret['date']=dlret['dlstdt']+MonthEnd(0)

        #merge delisted return with return
        crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','date'])
        crsp['dlret']=crsp['dlret'].fillna(0)

        crsp['ret']=crsp['ret'].fillna(0)
        
        # retadj factors in the delisting returns
        crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

        crsp.loc[crsp.prc<0,'prc']=abs(crsp['prc'])

        # calculate market equity
        crsp['me']=crsp['prc'].abs()*crsp['shrout'] 
        crsp=crsp.drop(['dlret','dlstdt'], axis=1)
        crsp=crsp.sort_values(by=['date','permco','me'])

        # ### Aggregate Market Cap ###
        # # sum of me across different permno belonging to same permco a given date
        # crsp_summe = crsp.groupby(['date','permco'])['me'].sum().reset_index()

        # # largest mktcap within a permco/date
        # crsp_maxme = crsp.groupby(['date','permco'])['me'].max().reset_index()

        # # join by jdate/maxme to find the permno
        # crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['date','permco','me'])

        # # drop me column and replace with the sum me
        # crsp1=crsp1.drop(['me'], axis=1)

        # # join with sum of me to get the correct market cap info
        # crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['date','permco'])

        # # sort by permno and date and also drop duplicates
        # crsp2=crsp2.sort_values(by=['permno','date']).drop_duplicates()


        return crsp
    
    def get_daily_data(self):
        
        crsp_d=self.Conn.raw_sql(f''' select a.permco, a.permno, a.date, a.PRC, a.shrout, a.ret, a.vol, b.shrcd, b.exchcd, b.siccd, b.ticker
                            from crsp.dsf as a
                            left join crsp.msenames as b
                            on a.permno=b.permno 
                            and a.date>=b.namedt
                            and b.nameendt>=a.date
                            where a.date between '{self.begin}' and '{self.end}'
                            AND shrcd in ('10','11')
                                                ''',date_cols=['date'])  
        
        # change variable format to int
        crsp_d= crsp_d.dropna(subset=['permco','permno','shrcd','exchcd','siccd'])

        crsp_d[['permco','permno','shrcd','exchcd','siccd']]=\
            crsp_d[['permco','permno','shrcd','exchcd','siccd']].astype(int)

        # add delisting return
        dlret = self.Conn.raw_sql("""
                            select permno, dlret, dlstdt 
                            from crsp.msedelist
                            """, date_cols=['dlstdt'])

        dlret.permno=dlret.permno.astype(int)

        dlret.rename(columns={'dlstdt':'date'}, inplace= True)

        #merge delisted return with return
        crsp = pd.merge(crsp_d, dlret, how='left',on=['permno','date'])
        crsp['dlret']=crsp['dlret'].fillna(0)
        crsp['ret']=crsp['ret'].fillna(0)
        # retadj factors in the delisting returns
        crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

        crsp.loc[crsp.prc<0,'prc']=abs(crsp['prc'])

        # calculate market equity
        crsp['me']=crsp['prc'].abs()*crsp['shrout'] 
        crsp=crsp.drop(['dlret'], axis=1)
        crsp=crsp.sort_values(by=['date','permco','me'])

        ### Aggregate Market Cap ###
        # sum of me across different permno belonging to same permco a given date
        crsp_summe = crsp.groupby(['date','permco'])['me'].sum().reset_index()

        # largest mktcap within a permco/date
        crsp_maxme = crsp.groupby(['date','permco'])['me'].max().reset_index()

        # join by jdate/maxme to find the permno
        crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['date','permco','me'])

        # drop me column and replace with the sum me
        crsp1=crsp1.drop(['me'], axis=1)

        # join with sum of me to get the correct market cap info
        crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['date','permco'])

        # sort by permno and date and also drop duplicates
        crsp2=crsp2.sort_values(by=['permno','date']).drop_duplicates()


        return crsp2

In [57]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [58]:
import wrds

In [59]:
Conn=wrds.Connection()

Loading library list...
Done


In [60]:
# capital market block 
crsp=stock('01/01/1996','01/01/2023',Conn).get_monthly_data()
# adjusted scale
crsp['shrout']=crsp['shrout']*1000
# only year end value
crsp=crsp.sort_values(['permno','year'])

In [61]:
# financial statement block
# IB Income Before Extraordinary #18
# AT Total assest
# APALCH  Accounts Payable and Accrued Liabilities –Increase (Decrease) #304
# sale Sales (Net)
# PPENT Property, Plant and Equipment – Total (Net)
# OANCF Operating Activities – Net Cash Flow (Statement of Cash Flows)
comp=Conn.raw_sql('''
                  select gvkey, cusip,fyear, datadate, IB, AT, APALCH, sale, PPENT, OANCF, ni , DLTT, sich
                  from comp.funda
                  where 
                    indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate between '01/01/1996' and '12/31/2022'   
                            ''',date_cols=['datadate'])

comp.rename(columns={'datadate':'date'},inplace=True)
comp['date']=comp['date']+MonthEnd(0)
comp['year']=comp['date'].dt.year


In [62]:
#remove company with non-postive assests
comp=comp[comp['at']>1]  #caution 
#adjust companies that change financial reporting date
comp=comp.drop_duplicates(['gvkey','fyear'],keep='last')
comp=comp.sort_values(['gvkey','fyear'])

In [63]:
comp['oancf']=comp['oancf'].fillna(0)   #caution 
comp['ACCR']=comp['ib']-comp['oancf']
comp['dREV']=comp.groupby('gvkey')['sale'].diff()

In [64]:
#Link table for merge Compustat and Crsp
Link=Conn.raw_sql('''
                  select * from crsp.ccmxpf_linktable 
                  where (linktype ='LC' or linktype='LU')
                  and (linkprim ='C' or linkprim='P')
                                    ''',date_cols=['linkdt', 'linkenddt'])

Link['linkenddt']=Link['linkenddt'].fillna(pd.to_datetime('today')).dt.date
Link[['lpermno','lpermco','usedflag']]=\
    Link[['lpermno','lpermco','usedflag']].astype(int)
Link.rename(columns={'lpermno':'permno','lpermco':'permco'},inplace=True)

#Merge LINK
merge_comp=pd.merge(comp,Link,how='left',on=['gvkey'])
merge_comp=merge_comp[(merge_comp['date']>=merge_comp['linkdt']) & (merge_comp['date']<=merge_comp['linkenddt'])\
                      .drop(columns=['linktype', 'linkdt', 'linkenddt'])]
merge_comp

,gvkey,cusip,fyear,date,ib,at,apalch,sale,ppent,oancf,ni,dltt,sich,year,ACCR,dREV,linkprim,liid,linktype,permno,permco,usedflag,linkdt,linkenddt
0,001004,000361105,1995.000,1996-05-31,16.012,437.846,NaN,504.990,54.831,24.760,16.012,118.292,5080.000,1996,-8.748,NaN,P,01,LU,54594.000,20000.000,1.000,1972-04-24,2023-10-29
1,001004,000361105,1996.000,1997-05-31,23.025,529.584,NaN,589.328,71.108,9.531,23.025,116.818,5080.000,1997,13.494,84.338,P,01,LU,54594.000,20000.000,1.000,1972-04-24,2023-10-29
2,001004,000361105,1997.000,1998-05-31,35.657,670.559,NaN,782.123,82.905,22.823,35.657,177.509,5080.000,1998,12.834,192.795,P,01,LU,54594.000,20000.000,1.000,1972-04-24,2023-10-29
3,001004,000361105,1998.000,1999-05-31,41.671,726.630,NaN,918.036,104.012,28.525,41.671,180.939,5080.000,1999,13.146,135.913,P,01,LU,54594.000,20000.000,1.000,1972-04-24,2023-10-29
4,001004,000361105,1999.000,2000-05-31,35.163,740.998,NaN,1024.333,110.003,10.051,35.163,180.447,5080.000,2000,25.112,106.297,P,01,LU,54594.000,20000.000,1.000,1972-04-24,2023-10-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277923,349972,45339J105,2022.000,2022-12-31,-14.323,28.064,NaN,0.000,0.081,-13.078,-14.323,0.000,2836.000,2022,-1.245,0.000,P,01,LC,15642.000,55395.000,1.000,2021-08-04,2023-10-29
277927,350681,37428A103,2021.000,2021-12-31,85.469,10595.813,2304.169,512.094,117.975,-177.872,85.469,7.499,7374.000,2021,263.341,70.740,P,90,LC,22205.000,58855.000,1.000,2021-10-22,2023-10-29
277929,351038,74907L300,2021.000,2021-12-31,-21.463,9.357,2.045,0.000,0.000,-5.720,-21.463,0.000,2834.000,2021,-15.743,0.000,P,90,LC,16161.000,55612.000,1.000,2021-10-29,2023-10-29
277930,351038,74907L300,2022.000,2022-12-31,-9.381,14.458,-0.218,0.000,0.000,-8.481,-9.381,0.000,2834.000,2022,-0.900,0.000,P,90,LC,16161.000,55612.000,1.000,2021-10-29,2023-10-29


In [65]:
###Merge CSRP Data
Data=pd.merge(merge_comp,crsp,\
     how='inner',left_on=['permco','permno','date','year'],right_on=['permco','permno','date','year'])
###combine sich and sic 
Data['sic'] = np.where(Data['sich']>0, Data['sich'], Data['siccd'])
Data['sic']=Data['sic'].astype(int)
Data['TwoDigSIC'] = Data['sic'].astype(str).str[:2].astype(int)

In [66]:
# screening 
print(f'obs={len(Data)}')
Data=Data[~((Data['sic']>=6000)&(Data['sic']<=6199))] # banking firm 
print(f'obs={len(Data)}')
Data=Data[~((Data['sic']>=6300)&(Data['sic']<=6411))] # insurance firm 
print(f'obs={len(Data)}')

obs=124001
obs=109060
obs=105317


In [67]:
# Industries with less than eight observations are dropped from the sample.
Data['count']=1
Data['INDcounts']=Data.groupby(['TwoDigSIC','fyear'])['count'].transform(np.nansum)
Data=Data[Data['INDcounts']>=8]
print(f'obs={len(Data)}')

obs=104360


In [68]:
# at t-1
Data=Data.sort_values(['gvkey','fyear'])
Data['at_l1']=Data.groupby(['gvkey'])['at'].shift(1)

In [69]:
# accrual regression 
Data=Data.dropna(subset=['at_l1'])
Data['ACCR_at']=Data['ACCR']/Data['at_l1']
Data['dREV_at']=Data['dREV']/Data['at_l1']
Data['ppent_at']=Data['ppent']/Data['at_l1']

Data.describe()

,fyear,date,ib,at,apalch,sale,ppent,oancf,ni,dltt,sich,year,ACCR,dREV,permno,permco,usedflag,linkdt,prc,shrout,ret,vol,shrcd,exchcd,siccd,day,month,retadj,me,sic,TwoDigSIC,count,INDcounts,at_l1,ACCR_at,dREV_at,ppent_at
count,91685.000,91685,91627.000,91685.000,65149.000,91623.000,91502.000,91685.000,91627.000,91344.000,90562.000,91685.000,91627.000,91522.000,91685.000,91685.000,91685.000,91685,91135.000,91685.000,91685.000,91154.000,91685.000,91685.000,91685.000,91685.000,91685.000,91685.000,91135.000,91685.000,91685.000,91685.000,91685.000,91685.000,91627.000,91522.000,91502.000
mean,2007.701,2008-08-29 18:52:46.555052800,167.434,4558.582,20.455,2868.661,1209.909,355.420,168.075,1124.905,4567.451,2007.831,-188.211,163.417,63537.915,23255.306,1.000,1992-12-27 17:44:53.688171392,33.821,102742234.368,0.012,192919.084,10.996,2.249,4980.110,30.813,10.001,0.012,4457516.611,4587.593,45.486,1.000,240.930,4263.568,-0.086,0.109,0.278
min,1996.000,1996-06-30 00:00:00,-56121.900,1.015,-62084.000,-1964.999,0.000,-72362.000,-98696.000,0.000,100.000,1996.000,-71610.000,-172892.000,10001.000,7.000,1.000,1962-01-02 00:00:00,0.016,231000.000,-0.953,0.000,10.000,-2.000,0.000,28.000,1.000,-0.953,167.100,100.000,10.000,1.000,8.000,1.015,-42.434,-17.208,0.000
25%,2001.000,2001-12-31 00:00:00,-9.160,70.949,-1.828,47.964,6.929,-0.993,-9.616,0.149,2890.000,2001.000,-82.291,-2.539,37955.000,11182.000,1.000,1985-10-01 00:00:00,4.300,12146000.000,-0.071,8081.250,11.000,1.000,3443.000,31.000,9.000,-0.071,65756.000,2911.000,29.000,1.000,61.000,66.659,-0.117,-0.020,0.069
50%,2007.000,2007-12-31 00:00:00,4.208,331.726,0.614,272.277,48.579,16.468,4.153,24.808,3829.000,2007.000,-13.602,9.397,78213.000,16356.000,1.000,1995-02-01 00:00:00,13.360,29269000.000,0.002,37052.500,11.000,3.000,4210.000,31.000,12.000,0.002,332269.500,3841.000,38.000,1.000,148.000,300.815,-0.058,0.048,0.173
75%,2014.000,2014-12-31 00:00:00,56.000,1613.749,7.800,1334.154,339.332,133.372,56.605,376.849,5961.000,2014.000,-1.203,85.164,85625.000,35222.000,1.000,2000-10-23 00:00:00,31.150,69162000.000,0.076,131169.750,11.000,3.000,7311.000,31.000,12.000,0.076,1627519.833,6211.000,62.000,1.000,392.000,1475.264,-0.015,0.173,0.385
max,2022.000,2022-12-31 00:00:00,99803.000,1463988.000,42743.000,569962.000,259651.000,122151.000,99803.000,377138.000,9997.000,2022.000,79796.000,121983.000,93436.000,59426.000,1.000,2022-11-22 00:00:00,141600.000,16976763000.000,16.251,38816274.000,11.000,6.000,9999.000,31.000,12.000,16.251,2324390219.000,9999.000,99.000,1.000,952.000,1463988.000,27.188,177.329,55.865
std,7.732,NaN,1543.713,29878.628,455.634,14171.136,6758.794,2296.586,1587.606,6891.790,1989.236,7.707,1452.009,2421.368,28744.535,16969.022,0.000,NaN,860.119,395477828.524,0.208,718804.660,0.062,0.939,2324.452,0.455,3.376,0.208,28784838.860,1991.962,19.801,0.000,219.851,28393.343,0.414,0.955,0.443


In [107]:
# corporate governace block'
directorUS=Conn.raw_sql('''
                         select *
                        from RISK.RMDIRECTORS 
                        order by company_id, year
                       '''
                        ,date_cols=['MeetingDate'])
# tenure 
directorUS=directorUS.sort_values(['company_id','director_detail_id', 'year'])
directorUS['count']=1
directorUS['tenure']=directorUS.groupby(['company_id','director_detail_id'])['count'].cumsum()

# calculate independence
directorUS['outsiders']=0
directorUS.loc[(directorUS['classification']=='I-NED')|
               (directorUS['classification']=='I')
               ,'outsiders']=1

directorUS['insiders']=0
directorUS.loc[(directorUS['classification']=='E')|
               (directorUS['classification']=='Exec')|
               (directorUS['classification']=='NI-NED')
               ,'insiders']=1

directorUS['affiliate']=0
directorUS.loc[(directorUS['classification']=='L')
               ,'affiliate']=1

# audit committee
directorUS['audit_membership']=directorUS['audit_membership'].apply(lambda x: str(x).lower())
directorUS['acc_committee']=0
directorUS.loc[(directorUS['audit_membership']=='member')|
               (directorUS['classification']=='chair')
               ,'acc_committee']=1


In [71]:
# Get blockholder 
# It seems percent of voting power is simlar and have less outliers 
directorUS=pd.merge(directorUS,Data[['cusip','year','shrout']],how='inner',on=['cusip','year'])
directorUS['holding']=directorUS['num_of_shares']/directorUS['shrout']
directorUS['blockholder']=0
directorUS.loc[directorUS['holding']>=0.05,'blockholder']=1
directorUS['OUTblockholder']=0
directorUS.loc[(directorUS['holding']>=0.05)&(directorUS['outsiders']==1),'OUTblockholder']=1
# CEO holdings
directorUS['ceo']=0
directorUS.loc[(directorUS['employment_ceo']=='Yes'),'ceo']=1
directorUS['ceo_holding']=0
directorUS.loc[(directorUS['ceo']==1),'ceo_holding']=directorUS['num_of_shares']/directorUS['shrout']
# compress data to firm level
directorUS=directorUS.sort_values(['company_id', 'year'])
DirectorsAgg=directorUS.groupby(['cusip', 'year'])[['outsiders','insiders','ceo_holding','affiliate','count']].agg(np.nansum).reset_index()
DirectorsAgg['out']=DirectorsAgg['outsiders']/DirectorsAgg['count']
DirectorsAgg['ins']=DirectorsAgg['insiders']/DirectorsAgg['count']
DirectorsAgg['aff']=DirectorsAgg['affiliate']/DirectorsAgg['count']

DirectorsAgg['BD51']=0
DirectorsAgg.loc[DirectorsAgg['out']>0.5,'BD51']=1

DirectorsAgg['BD100']=0
DirectorsAgg.loc[DirectorsAgg['out']==1,'BD100']=1

# audit committee level 
auditcomittee=directorUS[directorUS['acc_committee']==1].groupby(['cusip', 'year'])[['outsiders','insiders','affiliate','OUTblockholder','count']].agg(np.nansum).reset_index()
auditcomittee['audit_out']=auditcomittee['outsiders']/auditcomittee['count']
auditcomittee['audit_ins']=auditcomittee['insiders']/auditcomittee['count']
auditcomittee['audit_aff']=auditcomittee['affiliate']/auditcomittee['count']

auditcomittee['Audit100']=0
auditcomittee.loc[auditcomittee['audit_out']==1,'Audit100']=1
auditcomittee['Audit51']=0
auditcomittee.loc[auditcomittee['audit_out']>0.5,'Audit51']=1


In [72]:
import docx

In [73]:
doc = docx.Document()

In [74]:
# Summary Table2
table2=pd.concat([
        pd.merge(DirectorsAgg.describe().loc['mean'][['out','ins','aff']].to_frame().rename(columns={'mean':'Whole board'}),
                auditcomittee.describe().loc['mean'][['audit_out','audit_ins','audit_aff']].to_frame().T.rename(columns={'audit_out':'out',
                                                                                                'audit_ins':'ins',
                                                                                                'audit_aff':'aff'}).T.rename(columns={'mean':'Audit committee'}),
                how='inner',left_index=True,right_index=True ),
        pd.merge(DirectorsAgg[['BD100','BD51']].describe().loc['mean'].to_frame().rename(columns={'mean':'Whole board'}),
        auditcomittee.describe().loc['mean'][['Audit100','Audit51']].to_frame().T.rename(columns={'Audit100':'BD100',
                                                                                        'Audit51':'BD51'}).T.rename(columns={'mean':'Audit committee'}),
        how='inner',left_index=True,right_index=True )
        ])


In [75]:
table2.index=['Insiders','Outsidersb','Affiliatesc','100% outside directors','Majority of outside directors']
table2=table2.round(2)

In [76]:
# Initialise the table
t = doc.add_table(rows=table2.shape[0]+1, cols=table2.shape[1]+1)

In [77]:
# add header and row name
for i in range(table2.shape[0]):
    cell=table2.index[i]
    t.cell(i+1, 0).text = str(cell)

for i in range(table2.shape[1]):
    cell=table2.columns[i]
    t.cell(0, i+1).text = str(cell)

In [78]:
# Add the body of the data frame to the table
for i in range(table2.shape[0]):
    for j in range(table2.shape[1]):
        cell = table2.iat[i, j]
        t.cell(i+1, j+1).text = str(cell)

In [79]:
# Save the Word doc
doc.save('table 3.docx')

In [80]:
firm=pd.merge(DirectorsAgg[['cusip','year','out','BD51','ceo_holding']],auditcomittee[['cusip','year','audit_out','OUTblockholder','Audit100','Audit51']], how='inner',on=['cusip','year'])
data1=pd.merge(Data,firm,how='left',on=['cusip','year'])

In [81]:
data1=pd.merge(data1,data1[data1['out'].notnull()]['gvkey'].drop_duplicates(),how='inner',on=['gvkey'])

In [82]:
#winsor by indusry & year
WIN_LIMIT=1
def preprossess(a):
    a=a.astype(np.float64)
    a[np.isinf(a)] = np.nan
    a=np.clip(a,np.nanpercentile(a,WIN_LIMIT),np.nanpercentile(a,100-WIN_LIMIT))
    return a
features_winsor=['ACCR_at','dREV_at','ppent_at']

IND_list=list(data1.TwoDigSIC.unique())
year_list=list(data1.fyear.unique())
data1=data1.sort_values(['gvkey','fyear'])
for code in IND_list:
    for year in year_list:
        for x in features_winsor:
            data1.loc[(data1['TwoDigSIC']==code) & (data1['fyear']==year),x]=preprossess(data1.loc[(data1['TwoDigSIC']==code) & (data1['fyear']==year),x])
        

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out

In [83]:
# computation block
est_table=pd.DataFrame(columns=['coeff_dREV_at','coeff_ppent_at','intercept','fyear','TwoDigSIC'])
for year in tqdm(year_list):
    for code in tqdm(IND_list):
        regression_data=data1[(data1['TwoDigSIC']==code)&(data1['fyear']==year)][['ACCR_at','dREV_at','ppent_at']].dropna()
        
        if len(regression_data)>=8:
            results = smf.ols('ACCR_at ~ dREV_at + ppent_at',
                    data=regression_data).fit()
            coeff_dic={'coeff_dREV_at':results.params['dREV_at'] ,
                        'coeff_ppent_at': results.params['ppent_at'],
                        'intercept': results.params['Intercept'],
                        'fyear': year,
                        'TwoDigSIC' : code}
            coeff_dic = pd.DataFrame([coeff_dic])
            est_table = pd.concat([est_table, coeff_dic], ignore_index=True)
        # else:
        #     print(code,year,len(regression_data))

100%|██████████| 27/27 [00:02<00:00, 11.44it/s]


In [84]:
Data2=pd.merge(data1,est_table,how='inner',on=['TwoDigSIC','fyear'])
# AAC
Data2['ACC']=Data2['ACCR_at']-(Data2['intercept']+Data2['dREV_at']*Data2['coeff_dREV_at']+Data2['ppent_at']*Data2['coeff_ppent_at'])
# abs AAC
Data2['ACC_abs']=abs(Data2['ACC'])



In [85]:
# lowbond=Data1['ACC_abs'].describe(percentiles=[0.001,0.999]).to_frame().T['1%'][0]
# highbond=Data1['ACC_abs'].describe(percentiles=[0.001,0.999]).to_frame().T['99%'][0]
# Data1=Data1[(Data1['ACC_abs']>lowbond )& (Data1['ACC_abs']<highbond) ]

In [86]:
# other control
Data2['MB']=Data2['me']*1000/(Data2['at_l1']*1000000)
Data2=Data2.sort_values(['gvkey','fyear']).reset_index().drop('index',axis=1)
Data2['NId']=Data2.groupby('gvkey')['ni'].diff()
Data2['NId']=abs(Data2['NId'])
Data2['loss_dummy']=0
Data2.loc[Data2['ni']<0,'loss_dummy']=1
Data2.groupby('gvkey')[['loss_dummy']].rolling(2,min_periods=1).sum()
Data2['NegNI_']=Data2.groupby('gvkey')['loss_dummy'].rolling(2,min_periods=1).agg(np.nansum).to_frame().reset_index()['loss_dummy']
Data2['NegNI']=0
Data2.loc[Data2['NegNI_']>=2,'NegNI']=1
Data2['debt']=Data2['dltt']/Data2['at_l1']
Data2['asset']=np.log(Data2['at']+1)
Data2['ib_at']=Data2['ib']/Data2['at_l1']
Data2['oancf_at']=Data2['oancf']/Data2['at_l1']

In [87]:
# AAAC
Data2=Data2.sort_values(['gvkey','fyear']).reset_index().drop('index',axis=1)

In [88]:
Data2['accr_std']=Data2[['gvkey','ACCR']].groupby('gvkey').transform(lambda x: x.rolling(10,min_periods=10).std().shift(1))

In [89]:
Data2['accr_std'].describe()

count   15658.000
mean      417.172
std      1700.416
min         0.390
25%        30.459
50%        80.151
75%       263.318
max     44247.900
Name: accr_std, dtype: float64

In [90]:
Data2['accr_std_Decile'] = pd.qcut(Data2['accr_std'],10,labels=[1,2,3,4,5,6,7,8,9,10])

In [91]:
Data2[Data2['accr_std_Decile'].notnull()].describe()

,fyear,date,ib,at,apalch,sale,ppent,oancf,ni,dltt,sich,year,ACCR,dREV,permno,permco,usedflag,linkdt,prc,shrout,ret,vol,shrcd,exchcd,siccd,day,month,retadj,me,sic,count,INDcounts,at_l1,ACCR_at,dREV_at,ppent_at,out,BD51,ceo_holding,audit_out,OUTblockholder,Audit100,Audit51,coeff_dREV_at,coeff_ppent_at,intercept,ACC,ACC_abs,MB,NId,loss_dummy,NegNI_,NegNI,debt,asset,ib_at,oancf_at,accr_std
count,15658.000,15658,15658.000,15658.000,10654.000,15658.000,15640.000,15658.000,15658.000,15599.000,15637.000,15658.000,15658.000,15658.000,15658.000,15658.000,15658.000,15658,15655.000,15658.000,15658.000,15655.000,15658.000,15658.000,15658.000,15658.000,15658.000,15658.000,15655.000,15658.000,15658.000,15658.000,15658.000,15658.000,15658.000,15640.000,12467.000,12467.000,12467.000,12467.000,12467.000,12467.000,12467.000,15658.000,15658.000,15658.000,15640.000,15640.000,15655.000,15658.000,15658.000,15658.000,15658.000,15599.000,15658.000,15658.000,15658.000,15658.000
mean,2014.244,2015-03-18 18:52:11.434410752,684.142,14397.287,67.111,9147.269,3946.379,1294.228,693.903,3570.736,4394.094,2014.387,-610.086,423.311,61024.804,18865.276,1.000,1985-12-12 12:50:12.798569472,62.522,245480666.624,0.012,437853.061,11.000,1.849,4506.968,30.822,9.916,0.012,15069142.669,4394.622,1.000,174.091,13650.665,-0.056,0.066,0.281,0.805,0.986,0.025,0.994,0.005,0.984,0.993,0.015,-0.070,-0.046,0.002,0.040,1.718,414.452,0.161,0.320,0.085,0.247,7.975,0.055,0.111,417.172
min,2006.000,2006-09-30 00:00:00,-43297.000,11.553,-8442.000,0.000,0.012,-45951.000,-38732.000,0.000,200.000,2006.000,-71610.000,-172892.000,10026.000,7.000,1.000,1962-01-02 00:00:00,0.198,1433000.000,-0.878,75.000,11.000,0.000,250.000,28.000,1.000,-0.878,5776.640,200.000,1.000,10.000,8.895,-0.797,-5.888,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.861,-9.970,-0.542,-0.705,0.000,0.004,0.000,0.000,0.000,0.000,0.000,2.530,-1.314,-1.505,0.390
25%,2010.000,2011-01-31 00:00:00,22.013,875.077,-10.388,790.438,116.362,71.205,22.001,68.517,3290.000,2011.000,-376.824,-23.150,37284.000,10256.000,1.000,1972-12-14 00:00:00,19.900,33925750.000,-0.042,49147.500,11.000,1.000,3350.250,31.000,9.000,-0.042,938293.113,3290.000,1.000,48.000,817.397,-0.086,-0.013,0.094,0.750,1.000,0.002,1.000,0.000,1.000,1.000,-0.049,-0.129,-0.066,-0.023,0.012,0.627,15.321,0.000,0.000,0.000,0.069,6.775,0.021,0.064,30.459
50%,2014.000,2014-12-31 00:00:00,109.303,2520.390,5.734,2156.162,449.275,235.667,110.939,556.000,3823.000,2014.000,-102.230,68.676,75905.000,16561.000,1.000,1990-02-12 00:00:00,37.990,70221000.000,0.011,131037.000,11.000,1.000,3827.000,31.000,12.000,0.011,2701924.461,3823.000,1.000,110.000,2361.436,-0.050,0.041,0.194,0.833,1.000,0.005,1.000,0.000,1.000,1.000,0.013,-0.054,-0.045,0.005,0.027,1.152,52.801,0.000,0.000,0.000,0.219,7.833,0.054,0.103,80.151
75%,2018.000,2018-12-31 00:00:00,436.875,8418.476,42.600,6476.709,1968.461,820.100,440.617,2288.557,5399.000,2018.000,-22.734,340.325,83621.000,21793.000,1.000,1996-10-29 00:00:00,67.515,183645750.000,0.062,379199.000,11.000,3.000,5621.000,31.000,12.000,0.062,9532463.168,5331.000,1.000,249.000,7911.164,-0.022,0.119,0.398,0.889,1.000,0.016,1.000,0.000,1.000,1.000,0.081,0.001,-0.021,0.031,0.052,2.039,209.298,0.000,0.000,0.000,0.351,9.038,0.098,0.154,263.318
max,2022.000,2022-12-31 00:00:00,99803.000,1463988.000,25062.000,569962.000,259651.000,122151.000,99803.000,265754.000,8742.000,2022.000,47100.000,121983.000,93436.000,59277.000,1.000,2022-05-17 00:00:00,5908.870,16976763000.000,1.329,38816274.000,11.000,6.000,9999.000,31.000,12.000,1.329,2324390219.000,8742.000,1.000,788.000,1463988.000,1.832,39.286,3.911,1.000,1.000,2.217,1.000,1.000,1.000,1.000,1.439,6.457,0.246,1.103,1.103,32.193,45480.000,1.000,2.000,1.000,4.710,14.197,2.604,1.550,44247.900
std,4.528,NaN,3102.118,53865.101,652.127,28588.727,13793.201,4610.175,3114.180,12280.854,1786.556,4.509,2487.476,4831.328,27281.940,12590.225,0.000,NaN,135.164,679141647.030,0.103,1149284.405,0.000,0.989,1875.144,0.441,3.468,0.10

In [92]:
Data2=Data2[Data2['fyear']>=2007]
Data2=Data2[Data2['accr_std_Decile'].notnull()]

In [93]:
Data2['adjust']=Data2.groupby(['accr_std_Decile'])['ACC_abs'].transform(np.nanmedian)
Data2['AAAC']=Data2['ACC_abs']-Data2['adjust']
# Data1['AAAC']=preprossess(Data1['AAAC'])

In [94]:
# Summary table 3
Data2[['ACC','ACC_abs','AAAC','ACCR_at']].describe(percentiles=[0.001,0.999])

,ACC,ACC_abs,AAAC,ACCR_at
count,15554.000,15554.000,15554.000,15572.000
mean,0.002,0.040,0.013,-0.056
std,0.062,0.047,0.047,0.074
min,-0.705,0.000,-0.031,-0.797
0.1%,-0.374,0.000,-0.031,-0.499
50%,0.005,0.027,0.000,-0.050
99.9%,0.326,0.423,0.393,0.337
max,1.103,1.103,1.072,1.832


In [95]:
Data2.describe()

,fyear,date,ib,at,apalch,sale,ppent,oancf,ni,dltt,sich,year,ACCR,dREV,permno,permco,usedflag,linkdt,prc,shrout,ret,vol,shrcd,exchcd,siccd,day,month,retadj,me,sic,count,INDcounts,at_l1,ACCR_at,dREV_at,ppent_at,out,BD51,ceo_holding,audit_out,OUTblockholder,Audit100,Audit51,coeff_dREV_at,coeff_ppent_at,intercept,ACC,ACC_abs,MB,NId,loss_dummy,NegNI_,NegNI,debt,asset,ib_at,oancf_at,accr_std,adjust,AAAC
count,15572.000,15572,15572.000,15572.000,10599.000,15572.000,15554.000,15572.000,15572.000,15513.000,15551.000,15572.000,15572.000,15572.000,15572.000,15572.000,15572.000,15572,15569.000,15572.000,15572.000,15569.000,15572.000,15572.000,15572.000,15572.000,15572.000,15572.000,15569.000,15572.000,15572.000,15572.000,15572.000,15572.000,15572.000,15554.000,12421.000,12421.000,12421.000,12421.000,12421.000,12421.000,12421.000,15572.000,15572.000,15572.000,15554.000,15554.000,15569.000,15572.000,15572.000,15572.000,15572.000,15513.000,15572.000,15572.000,15572.000,15572.000,15572.000,15554.000
mean,2014.289,2015-04-04 00:00:33.290521344,685.472,14444.920,67.092,9150.625,3956.770,1297.203,695.336,3584.645,4390.914,2014.428,-611.730,421.513,61043.858,18897.177,1.000,1985-12-17 07:52:49.072694528,62.706,245316892.564,0.012,437709.519,11.000,1.848,4504.249,30.823,9.956,0.012,15102001.150,4391.448,1.000,173.822,13695.640,-0.056,0.066,0.281,0.806,0.986,0.025,0.994,0.005,0.984,0.993,0.014,-0.069,-0.046,0.002,0.040,1.715,416.328,0.162,0.321,0.085,0.248,7.980,0.055,0.111,418.631,0.027,0.013
min,2007.000,2007-06-30 00:00:00,-43297.000,11.553,-8442.000,0.000,0.012,-45951.000,-38732.000,0.000,200.000,2007.000,-71610.000,-172892.000,10026.000,7.000,1.000,1962-01-02 00:00:00,0.198,1433000.000,-0.878,96.000,11.000,0.000,250.000,28.000,1.000,-0.878,5776.640,200.000,1.000,10.000,8.895,-0.797,-5.888,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.861,-9.970,-0.542,-0.705,0.000,0.004,0.000,0.000,0.000,0.000,0.000,2.530,-1.314,-1.505,0.393,0.024,-0.031
25%,2010.000,2011-03-31 00:00:00,22.009,880.016,-10.408,793.432,117.126,71.627,22.000,70.954,3272.000,2011.000,-378.460,-23.650,37284.000,10288.000,1.000,1972-12-14 00:00:00,19.940,33960000.000,-0.042,49163.000,11.000,1.000,3350.000,31.000,9.000,-0.042,941150.253,3285.500,1.000,48.000,820.422,-0.086,-0.014,0.094,0.750,1.000,0.002,1.000,0.000,1.000,1.000,-0.049,-0.129,-0.066,-0.023,0.012,0.626,15.436,0.000,0.000,0.000,0.070,6.781,0.021,0.064,30.579,0.025,-0.015
50%,2014.000,2014-12-31 00:00:00,109.482,2534.547,5.762,2160.327,450.016,236.551,111.079,561.138,3823.000,2014.000,-102.324,68.622,75912.000,16587.000,1.000,1990-02-16 00:00:00,38.050,70201500.000,0.011,130842.000,11.000,1.000,3827.000,31.000,12.000,0.011,2707344.975,3823.000,1.000,110.000,2368.094,-0.050,0.040,0.193,0.833,1.000,0.005,1.000,0.000,1.000,1.000,0.013,-0.054,-0.045,0.005,0.027,1.147,53.006,0.000,0.000,0.000,0.219,7.838,0.054,0.103,80.422,0.027,0.000
75%,2018.000,2018-12-31 00:00:00,437.887,8462.250,42.710,6486.704,1975.478,820.950,442.129,2298.443,5331.000,2018.000,-22.997,340.492,83671.000,21793.250,1.000,1996-11-01 00:00:00,67.700,183291500.000,0.062,378371.000,11.000,3.000,5621.000,31.000,12.000,0.062,9547870.884,5331.000,1.000,249.000,7941.378,-0.022,0.119,0.399,0.889,1.000,0.016,1.000,0.000,1.000,1.000,0.081,0.001,-0.021,0.031,0.052,2.036,210.622,0.000,0.000,0.000,0.351,9.043,0.097,0.154,264.519,0.030,0.024
max,2022.000,2022-12-31 00:00:00,99803.000,1463988.000,25062.000,569962.000,259651.000,122151.000,99803.000,265754.000,8742.000,2022.000,47100.000,121983.000,93436.000,59277.000,1.000,2022-05-17 00:00:00,5908.870,16976763000.000,1.329,38816274.000,11.000,6.000,9999.000,31.000,12.000,1.329,2324390219.000,8742.000,1.000,788.000,1463988.000,1.832,39.286,3.911,1.000,1.000,2.217,1.000,1.000,1.000,1.000,1.439,6.457,0.246,1.103,1.103,32.193,45480.000,1.000,2.000,1.000,4.710,14.197,2.604,1.550,44247.900,0.031,1.072
std,4.499,NaN,3108.805,53994.522,653.090,28531.737,13811.570,4619.473,3121.107,12310.474,1786.762,4.488,2493.222,4837.116,2

In [96]:
Data2=Data2.drop(['usedflag','linkdt','linkenddt'],axis=1)

In [97]:
data3=Data2[['permno','fyear','AAAC','ACC_abs','ACC','ACCR_at','out','BD51', 'ceo_holding', 'audit_out', 'OUTblockholder',
       'Audit100', 'Audit51','NegNI', 'debt', 'asset','MB', 'loss_dummy','NId','ib_at','oancf_at','TwoDigSIC','at_l1','at']]

In [98]:
data3['TwoDigSIC']=data3['TwoDigSIC'].astype(int)

/var/folders/v9/7f1j8bln1qx23fwm9dywqccm0000gn/T/ipykernel_93410/1457674687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['TwoDigSIC']=data3['TwoDigSIC'].astype(int)


In [99]:
data3['Non_discretionary_accrul']=data3['ACCR_at']-data3['ACC']

/var/folders/v9/7f1j8bln1qx23fwm9dywqccm0000gn/T/ipykernel_93410/3266049080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['Non_discretionary_accrul']=data3['ACCR_at']-data3['ACC']


In [100]:
data3.dropna(subset=['out']).to_stata('data_for_replication.dta')

In [101]:
data3.describe()

,permno,fyear,AAAC,ACC_abs,ACC,ACCR_at,out,BD51,ceo_holding,audit_out,OUTblockholder,Audit100,Audit51,NegNI,debt,asset,MB,loss_dummy,NId,ib_at,oancf_at,TwoDigSIC,at_l1,at,Non_discretionary_accrul
count,15572.000,15572.000,15554.000,15554.000,15554.000,15572.000,12421.000,12421.000,12421.000,12421.000,12421.000,12421.000,12421.000,15572.000,15513.000,15572.000,15569.000,15572.000,15572.000,15572.000,15572.000,15572.000,15572.000,15572.000,15554.000
mean,61043.858,2014.289,0.013,0.040,0.002,-0.056,0.806,0.986,0.025,0.994,0.005,0.984,0.993,0.085,0.248,7.980,1.715,0.162,416.328,0.055,0.111,43.491,13695.640,14444.920,-0.059
std,27290.273,4.499,0.047,0.047,0.062,0.074,0.106,0.118,0.073,0.055,0.070,0.125,0.086,0.279,0.246,1.687,1.997,0.368,1669.617,0.111,0.100,17.786,52037.623,53994.522,0.042
min,10026.000,2007.000,-0.031,0.000,-0.705,-0.797,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.530,0.004,0.000,0.000,-1.314,-1.505,13.000,8.895,11.553,-0.431
25%,37284.000,2010.000,-0.015,0.012,-0.023,-0.086,0.750,1.000,0.002,1.000,0.000,1.000,1.000,0.000,0.070,6.781,0.626,0.000,15.436,0.021,0.064,32.000,820.422,880.016,-0.077
50%,75912.000,2014.000,0.000,0.027,0.005,-0.050,0.833,1.000,0.005,1.000,0.000,1.000,1.000,0.000,0.219,7.838,1.147,0.000,53.006,0.054,0.103,38.000,2368.094,2534.547,-0.056
75%,83671.000,2018.000,0.024,0.052,0.031,-0.022,0.889,1.000,0.016,1.000,0.000,1.000,1.000,0.000,0.351,9.043,2.036,0.000,210.622,0.097,0.154,53.000,7941.378,8462.250,-0.037
max,93436.000,2022.000,1.072,1.103,1.103,1.832,1.000,1.000,2.217,1.000,1.000,1.000,1.000,1.000,4.710,14.197,32.193,1.000,45480.000,2.604,1.550,87.000,1463988.000,1463988.000,0.729
